In [ ]:
%pip install llama-parse


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. LLalmaParser 사용하기

In [7]:
# 필요 모듈 import 및 env 설정
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
# 비동기 처리 문제 해결
import nest_asyncio
nest_asyncio.apply()

In [9]:
# LlamaParse 파서 설정 ("markdown"과 "text" 형식 중 선택 가능)
parser = LlamaParse(result_type="markdown")

# SimpleDirectoryReader를 사용하여 파일 파싱
file_extractor = {".pdf": parser}  # PDF 파일을 LlamaParse로 처리하도록 설정
documents = SimpleDirectoryReader(
    input_files=["./data/240510_카카오_AI투자,눈에 밟히는 비용_유진투자증권.pdf"], file_extractor=file_extractor
).load_data()

Started parsing the file under job_id 7aad9b19-6f08-4293-8f96-aaa69d59d9b6
......

In [10]:
print(documents[0])

Doc ID: ba75f0de-f4ca-4549-a3bf-2df28d5c038d
Text: Korea | 인터넷 | 2024.05.10  # 카카오  # 투자의견  BUY(유지)  # 목표주가  73,000
원(유지)  # 현재주가  48,600 원(05/09)  # 시가총액  21,639(십억원)  # 인터넷/게임/우주  정의훈_
02)368-6170_ uihoon0607@eugenefn.com  # 1분기 실적  - 매출액 1조
9,884억원(+22.5%yoy), 영업이익 1,203억원(+92.2%yoy), 컨센서스(1,271억원) 소폭 하회 - 톡비즈
광고 매출은 메시지 비즈니스와 비즈보드에서 모두 두 자릿수 성장률 기록하며 성장. 중국 이커머스 플랫폼의 국내 마케팅 확장
또한 동사 광고 성장에 일...


In [11]:
file_name = "parse_kakao_doc1.md"
with open(file_name, 'w', encoding='utf-8') as file:
    for n, doc in enumerate(documents, start=1):
        file.write(f'#####{n} page 요약\n')
        file.write(doc.text+'\n')

In [12]:
'''
document 생성시 instruction promprt 사용하기
'''
documents_with_instruction = LlamaParse(
    result_type='markdown',
    prompt="""
    전체 내용을 투자자 관점에서 간결하게 설명해주시기 바랍니다.
    문서 안에 표가 포함 된 경우 수치를 포함해 요약해주세요.
    """
).load_data("./data/240510_카카오_AI투자,눈에 밟히는 비용_유진투자증권.pdf")

Started parsing the file under job_id 355fa835-0fd8-4666-962a-6308a9e2c820


In [13]:
file_name = "parse_kakao_doc2.md"
with open(file_name, 'w', encoding='utf-8') as file:
    for n, doc in enumerate(documents_with_instruction, start=1):
        file.write(f'#####{n} page 요약\n')
        file.write(doc.text+'\n')
        

### 2. LlamaParser를 활용한 RAG 시스템 구축

In [14]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
Settings.llm = OpenAI(model="gpt-4o", temperature=0.5)
embedding_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [ ]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(
    documents=documents_with_instruction,
    embed_model = embedding_model,
    show_progres = True,
)
index.storage_context.persist(persist_dir="./index/ch05_2_llamaparser_vector_index_storage") # index 저장

In [18]:
### 저장 된 인덱스 불러오기
from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="./index/ch05_2_llamaparser_vector_index_storage") 
index = load_index_from_storage(storage_context) # 밀집 검색 인덱스 로드

Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch05_2_llamaparser_vector_index_storage\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./index/ch05_2_llamaparser_vector_index_storage\index_store.json.


In [19]:
dense_retriever = index.as_retriever(
        similarity_top_k = 3,
)

In [ ]:
retrieval_results = dense_retriever.retrieve("카카오 2024년 실적 요약")

In [12]:
from llama_index.core.response.notebook_utils import display_source_node
for res_node in retrieval_results:
    display_source_node(res_node, source_length=500)

**Node ID:** 1614d2c8-7e35-47d0-b9bb-697127a8fcff<br>**Similarity:** 0.8831501543600708<br>**Text:** 카카오는 2024년 1분기 실적에서 매출 1조 9,884억 원(+22.5% yoy)과 영업이익 1,203억 원(+92.2% yoy)을 기록했으나, 영업이익은 시장 예상치인 1,271억 원과 당사 추정치 1,301억 원을 소폭 하회했습니다.

주요 성장 요인은 톡비즈 광고와 페이 부문으로, 특히 금융과 커머스 업종의 광고 활성화가 긍정적인 영향을 미쳤습니다. 반면, 콘텐츠 부문은 신작 효과에도 불구하고 기존 매출 감소로 어려움을 겪었고, 뮤직 부문도 아티스트 음반 판매 감소로 매출이 줄었습니다.

AI 투자와 관련하여 카카오는 약 2억 달러 규모의 교환사채를 발행했으며, 이 자금의 일부는 AI 경쟁력 강화에 사용될 예정입니다. 그러나 이러한 투자로 인해 카카오브레인의 영업적자가 확대될 것으로 예상되며, 단기적으로는 수익성에 부정적인 영향을 미칠 것으로 보입니다.<br>

**Node ID:** ba59882e-736b-4dce-b8e0-4d86b7dc9e5f<br>**Similarity:** 0.8610891056103033<br>**Text:** ### 카카오 실적 요약

#### 1. 분기 실적 (2023년 1Q - 2024년 4Q 예상)
- **매출액**:
- 2023년: 7,557억 원
- 2024년 예상: 8,443억 원
- 연평균 성장률: 11.1% (2023), 11.7% (2024)

- **영업이익**:
- 2023년: 460.9억 원
- 2024년 예상: 581.8억 원
- 영업이익률(OPM): 6.1% (2023), 6.9% (2024)

- **당기순이익**:
- 2023년: -649.4억 원 (적자)
- 2024년 예상: 448.5억 원 (흑자 전환)

#### 2. 연간 실적 (2021 - 2024E)
- **매출액**:
- 2021: 6,136.7억 원
- 2022: 6,799.0억 원
- 2023: 7,557.0억 원
- 2024E: 8,443.3억 원

- **영업이익**:
- 2021: 594.9억 원
- 2022: 569.4억 원
- 2023: 460.9억 원
- 2024E: 581.8억...<br>

**Node ID:** 1a92934e-1109-4f36-a6ee-c277c37ea71f<br>**Similarity:** 0.8376008748418426<br>**Text:** ### 카카오 투자 요약

- **투자의견**: BUY (유지)
- **목표주가**: 73,000 원 (유지)
- **현재주가**: 48,600 원 (2024.05.09)
- **시가총액**: 21,639 억원

#### 1분기 실적
- **매출액**: 1조 9,884억원 (+22.5% YoY)
- **영업이익**: 1,203억원 (+92.2% YoY, 컨센서스 1,271억원 소폭 하회)

#### 주요 부문 성과
- **톡비즈 광고**: 두 자릿수 성장률 기록
- **플랫폼 기타 부문**: 페이 부문에서 역대 최고 매출
- **콘텐츠 부문**: 게임 매출 감소, 픽코마 및 스토리 매출 부진
- **뮤직 매출**: 아티스트 음반 판매 감소로 전분기 대비 감소

#### 비용 및 투자
- AI 투자 확대에 따른 영업적자 증가 예상
- 2,930억원의 교환사채 발행 자금 중 1,000억원은 AI 투자에 사용 예정

#### 주가 및 재무 지표
- **52주 최고가**: 61,900...<br>

In [13]:
query_engine = index.as_query_engine()

In [14]:
response = query_engine.query("카카오 2024년 실적 요약")

In [15]:
print(response)

카카오의 2024년 실적 요약은 다음과 같습니다:
- 매출액: 8,443억 원
- 영업이익: 581.8억 원
- 당기순이익: 448.5억 원
- EPS: 1,008원
- PER: 48.2배
